In [6]:
#Some basic settings and directory specification
#Ensure these match companion Matlab script (thompsonreplication_mainscript.m)
maindatadir='/Volumes/ExternalDriveD/Thompson_savedarrays/';
y1=1961;y2=2023;
numyrs=y2-y1+1;

numregs=237;

modelnames=['canesm5','miroc6','mpige'];
canesm5_enssz=50;miroc6_enssz=10;mpige_enssz=50;

# 1961-2023 global mean surface temp (from NASA GISS: https://climate.nasa.gov/vital-signs/global-temperature/)
GMST = [0.06,0.03,0.05,-0.20,-0.11,-0.06,-0.02,-0.08,0.05,0.03,-0.08,0.01,0.16,-0.07,-0.01,
        -0.10,0.18,0.07,0.16,0.26,0.32,0.14,0.31,0.16,0.12,0.18,0.32,0.39,0.27,0.45,
        0.41, 0.22, 0.23, 0.31, 0.45, 0.33, 0.46, 0.61, 0.38, 0.39, 0.54, 0.63, 
        0.62, 0.53, 0.68, 0.64, 0.66, 0.54, 0.66, 0.72, 0.61, 0.65, 0.68, 0.74, 0.90, 
        1.01, 0.92, 0.85, 0.98, 1.01, 0.85, 0.89, 1.17];
GMST_yr = np.arange(y1, y2+1, 1);

GMST_since1981=GMST[20:];


domasking=0; #original default is 1 -- only don't do if wanting to see all regions' data, or if MERRA2 processing hasn't yet been done
maskingvs='jra55'; #'merra2' or 'jra55'

#Set loop options
domainprep=1; #needed upon start-up; runtime 4 min
maket23fig2=0;
maket23fig3=0;
rerunmodelcomp=0; #about 40 min total; needed only if changing GEV set-up
convertvalsformapping=0; #NOW CREATED IN MATLAB
maket23fig4=0; #Figure 5; NOW CREATED IN MATLAB

In [2]:
#Import packages
import sys
sys.path.insert(0, 'Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages');
#sys.path.insert(0, 'Users/craymond/LocallyInstalledSoftware');
import iris 
import iris.coord_categorisation as icc
from iris.coord_categorisation import add_season_membership
import numpy as np
import matplotlib.pyplot as plt
import iris.plot as iplt
import cartopy.crs as ccrs
import cartopy as cart
import glob
import matplotlib.cm as mpl_cm
from iris.util import equalise_attributes
import scipy.stats as sps
from scipy.stats import genextreme as gev
import random
import scipy.io
import xarray as xr
import netCDF4 as nc
from matplotlib import ticker, cm
from matplotlib.colors import LogNorm

In [3]:
#Various functions
def time_slice(cube, year1, year2):
    year_cons = iris.Constraint(time=lambda cell: year1 <= cell.point.year <= year2)
    return cube.extract(year_cons)

def cube_to_array(cube):
    return cube.data.reshape(np.shape(cube.data)[0]*np.shape(cube.data)[1])

def return_levels_plot(distribution_pdf, x_values):
    '''
    Calculates probability of return levels 
    '''
    chance = []
    return_level = []
    for i, _ in enumerate(distribution_pdf):
        width = x_values[1] - x_values[0]
        P = []
        for a, b in zip(distribution_pdf[i:-1], distribution_pdf[i+1:]):
            P.append(((a+b) / 2) * width) 
        chance.append(sum(P)*100)
        return_level.append(x_values[i])
    return return_level, chance

def plot_points(axs_sel, data_array):
    obs_sort = np.sort(data_array)
    chance = []
    for i in range(len(obs_sort)):
        chance.append(((i+1) / len(obs_sort)) * 100)
    ret_per = []
    for each in chance:
        ret_per.append(100.*1/each)
    ret_per.reverse()
    axs_sel.plot(ret_per, obs_sort, '+b', alpha=.5, label='Observed TXx')

def plot_gev(axs_sel, data_array, min_val, max_val):
    shape, loc, scale = gev.fit(data_array)
    #print('shape: ',str(shape))
    #print('loc: ',str(loc))
    #print('scale: ',str(scale))
    x_val = np.linspace(min_val, max_val, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    ret_lev, chance = return_levels_plot(dist_pdf, x_val)
    chance2 = []
    for each in chance[:-1]:
        chance2.append(100.*1/each)
    axs_sel.plot(chance2[10:], ret_lev[10:-1], color='dodgerblue', label='GEV fit')
    
def bootstrap_uncertainties(axs_sel, data_array):
    # measure of uncertainty in distribution
    # 100 bootstraps, plot 5th-95th% range
    chance_list = []
    for i in np.arange(100):
        bootstrap_data = random.choices(data_array, k=len(data_array)) # allows double dipping
        shape, loc, scale = gev.fit(bootstrap_data)
        x_val = np.linspace(np.min(data_array)-.5, np.max(data_array)+2, 1000)
        dist_pdf = gev.pdf(x_val, shape, loc, scale)
        ret_lev, chance = return_levels_plot(dist_pdf, x_val)
        chance_list.append(chance)
    chance_5 = []
    chance_95 = []
    for i in np.arange(1000):
        new_val = []
        for each in chance_list:
            new_val.append(each[i])
        chance_5.append(np.sort(new_val)[5])
        chance_95.append(np.sort(new_val)[95])
    chanceyear_5 = []
    chanceyear_95 = []
    for i, each in enumerate(chance_5[:-1]):
        chanceyear_5.append(100.*1/each)
        chanceyear_95.append(100.*1/chance_95[i])
    axs_sel.plot(chanceyear_5[10:], ret_lev[10:-1], '--', color='dodgerblue')
    axs_sel.plot(chanceyear_95[10:], ret_lev[10:-1], '--', color='dodgerblue', label='Uncertainty (5th-95th%)')

def one_in_tenthousand(data_array):
    shape, loc, scale = gev.fit(data_array)
    x_val = np.linspace(np.min(data_array)-.5, np.max(data_array)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    ret_lev, chance = return_levels_plot(dist_pdf, x_val)
    return ret_lev[np.abs(np.asarray(chance)-0.01).argmin()]

def how_much_higher(data_array):
    mod = one_in_tenthousand(data_array)
    obs = np.max(data_array)
    return mod-obs

def plot_data_GMST(axs_sel, ann_max, GMST):
    axs_sel.plot(GMST, ann_max, 'b+')
    a, b = np.polyfit(GMST, ann_max, 1)
    bestfit = []
    for each in np.sort(GMST): bestfit.append(a*each+b)
    axs_sel.plot([np.min(GMST), np.max(GMST)], [np.min(bestfit), np.max(bestfit)], color='dodgerblue')
    #axs_sel.set_ylabel('TXx')
    #axs_sel.set_xlabel('GMST')

def adjust_obs(ann_max, GMST):
    offset = []
    a, b = np.polyfit(GMST, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b)) #1*a+b means adjusted for world with 1deg warming  
    return offset

def plot_EVT(axs_sel, data_array):
    axs_sel.set_xscale('log')    
    plot_points(axs_sel, data_array)
    plot_gev(axs_sel, data_array, np.min(data_array)-.5, np.max(data_array)+2)

#Loads main dataset
def load_annmax(reg,var): #var can be 'tw' or 't'
    return np.loadtxt(maindatadir+'era5output_'+var+'_regions.txt',delimiter=',')[reg,:]; 

def remove_max(ann_max, GMST):
    offset = adjust_obs(ann_max, GMST)
    ' Remove the max value from region, and corresponding GMST value '
    idx_max = np.where(offset == np.max(offset))[0][0]
    ann_max_new = np.delete(ann_max, idx_max)
    GMST_new = np.delete(GMST, idx_max)
    return ann_max_new, GMST_new

def one_in_hundred(data_array):
    shape, loc, scale = gev.fit(data_array)
    x_val = np.linspace(np.min(data_array)-.5, np.max(data_array)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    ret_lev, chance = return_levels_plot(dist_pdf, x_val)
    return ret_lev[np.abs(np.asarray(chance)-1).argmin()]

def plot_data_GMST(axs_sel, ann_max, GMST):
    axs_sel.plot(GMST, ann_max, '+', label='Observed TXx')
    a, b = np.polyfit(GMST, ann_max, 1)
    bestfit = []
    for each in np.sort(GMST): bestfit.append(a*each+b)
    axs_sel.plot([np.min(GMST), np.max(GMST)], [np.min(bestfit), np.max(bestfit)], label='Fit')
    
from scipy.interpolate import RegularGridInterpolator
def regrid(data, out_x, out_y):
    m = max(data.shape[0], data.shape[1])
    y = np.linspace(0, 1.0/m, data.shape[0])
    x = np.linspace(0, 1.0/m, data.shape[1])
    interpolating_function = RegularGridInterpolator((y, x), data)

    yv, xv = np.meshgrid(np.linspace(0, 1.0/m, out_y), np.linspace(0, 1.0/m, out_x))

    return interpolating_function((xv, yv))

In [26]:
#More functions
def offset_higher(reg,var):
    ''' loads data, 
    applies offset to present day, 
    calcs difference between observed max and 1-in-10000 event'''
    #var can be 't', 'tw', or 'twdatasetmean' (the latter being an ERA5/JRA55 mean)
    if var=='t' or var=='tw':
        ann_max = load_annmax(reg,var)
    elif var=='twdatasetmean':
        tmp_era5=np.loadtxt(maindatadir+'era5output_tw_regions.txt',delimiter=',')[reg,:];
        tmp_jra55=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        ann_max=(tmp_era5+tmp_jra55)/2;
    elif var=='tw_jra55':
        ann_max=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        
    # Adjust ann max to distance from best fit
    offset = []
    a, b = np.polyfit(GMST, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b))  # np.mean to make 'present day'
    val = how_much_higher(offset)
    return val

def offset_higher_removeevent(reg,var):
    ''' loads data, 
    applies offset to present day, 
    calcs difference between observed max and 1-in-10000 event'''
    #var can be 't', 'tw', or 'twdatasetmean' (the latter being an ERA5/JRA55 mean)
    if var=='t' or var=='tw':
        ann_max = load_annmax(reg,var)
    elif var=='twdatasetmean':
        tmp_era5=np.loadtxt(maindatadir+'era5output_tw_regions.txt',delimiter=',')[reg,:];
        tmp_jra55=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        ann_max=(tmp_era5+tmp_jra55)/2;
    elif var=='tw_jra55':
        ann_max=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        
    offset = []
    a, b = np.polyfit(GMST, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b))  #adjust for world with 1 deg of warming
    ann_max_new, GMST_new = remove_max(ann_max, GMST)
    # Adjust ann max to distance from best fit
    offset = []
    a, b = np.polyfit(GMST_new, ann_max_new, 1)
    for i, each in enumerate(ann_max_new):
        actual = each
        predicted = GMST_new[i]*a +b
        offset.append(actual-predicted+(1*a+b))  # np.mean to make 'present day'
    tenthos = one_in_tenthousand(offset)
    offset_full = []
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset_full.append(actual-predicted+(1*a+b))
    max_val = np.max(offset_full)
    return tenthos - max_val

def obs_max_ret_per_removeevent(reg,var):
    ' Returns the highest event in terms of return period, yrs '
    #var can be 't', 'tw', or 'twdatasetmean' (the latter being an ERA5/JRA55 mean)
    if var=='t' or var=='tw':
        ann_max = load_annmax(reg,var)
    elif var=='twdatasetmean':
        tmp_era5=np.loadtxt(maindatadir+'era5output_tw_regions.txt',delimiter=',')[reg,:];
        tmp_jra55=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        ann_max=(tmp_era5+tmp_jra55)/2;
    elif var=='tw_jra55':
        ann_max=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        
    ann_max_new, GMST_new = remove_max(ann_max, GMST)
    offset_new = []
    a, b = np.polyfit(GMST_new, ann_max_new, 1)
    for i, each in enumerate(ann_max_new):
        actual = each
        predicted = GMST_new[i]*a +b
        offset_new.append(actual-predicted+(1*a+b)) #adjust for world with 1 deg of warming
        
    shape, loc, scale = gev.fit(offset_new) # EVT distribution without observed max
    x_val = np.linspace(np.min(offset_new)-.5, np.max(offset_new)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    
    offset_full = []
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset_full.append(actual-predicted+(1*a+b))
        
    max_val = np.max(offset_full) #observed max
    chance = []
    for i, _ in enumerate(dist_pdf):
        P = []
        for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
            P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
        chance.append(sum(P)*100)
    x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
    if x == 0:
        result_new = 99999
    else:
        result_new = 100.*1/x
    return result_new

def obs_max_ret_per(reg,var):
    ' Returns the highest event in terms of return period, yrs '
    #var can be 't', 'tw', or 'twdatasetmean' (the latter being an ERA5/JRA55 mean)
    if var=='t' or var=='tw':
        ann_max = load_annmax(reg,var)
    elif var=='twdatasetmean':
        tmp_era5=np.loadtxt(maindatadir+'era5output_tw_regions.txt',delimiter=',')[reg,:];
        tmp_jra55=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        ann_max=(tmp_era5+tmp_jra55)/2;
    elif var=='tw_jra55':
        ann_max=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=',')[reg,:];
        
    offset = []
    a, b = np.polyfit(GMST, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b))  #adjust for world with 1 deg of warming
        
    max_val = np.max(offset)
    shape, loc, scale = gev.fit(offset) # EVT distribution
    x_val = np.linspace(np.min(offset)-.5, np.max(offset)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    chance = []
    for i, _ in enumerate(dist_pdf):
        P = []
        for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
            P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
        chance.append(sum(P)*100)
    x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
    if x == 0:
        result = 99999
    else:
        result = 100.*1/x
    return result

def offset_maxval(reg,var):
    ann_max = load_annmax(reg,var)
    offset = []
    a, b = np.polyfit(GMST, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b))  # np.mean to make 'present day'
    max_val = np.max(offset) # Maximum observed
    return max_val

def abs_max(reg,var):
    ann_max = load_annmax(reg,var)
    return np.max(ann_max)

def offset_100(reg,var):
    ann_max = load_annmax(reg,var)
    ann_max_new, GMST_new = remove_max(ann_max, GMST)
    offset = []
    a, b = np.polyfit(GMST_new, ann_max_new, 1)
    for i, each in enumerate(ann_max_new):
        actual = each
        predicted = GMST[i]*a +b
        offset.append(actual-predicted+(1*a+b))  # np.mean to make 'present day'
    shape, loc, scale = gev.fit(offset) # EVT distribution
    x_val = np.linspace(np.min(offset)-.5, np.max(offset)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    #dist_cdf = gev.cdf(x_val, shape, loc, scale)
    ret_lev, chance = return_levels_plot(dist_pdf, x_val)
    chance2 = []
    for each in chance[:-1]:
        chance2.append(100.*1/each)
    return ret_lev[(np.abs(np.asarray(chance2)-100)).argmin()]

In [7]:
#Troubleshooting the above
reg=116;var='tw';
ann_max = np.loadtxt(maindatadir+'era5output_'+var+'_regions.txt',delimiter=',')[reg,:]; 

#With observed max
offset = []
a, b = np.polyfit(GMST, ann_max, 1)
for i, each in enumerate(ann_max):
    actual = each
    predicted = GMST[i]*a +b
    offset.append(actual-predicted+(1*a+b))  #adjust for world with 1 deg of warming

#GEV fit with observed max
max_val = np.max(offset)
shape, loc, scale = gev.fit(offset) # EVT distribution
x_val = np.linspace(np.min(offset)-.5, np.max(offset)+2, 1000)
dist_pdf = gev.pdf(x_val, shape, loc, scale)
chance = []
for i, _ in enumerate(dist_pdf):
    P = []
    for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
        P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
    chance.append(sum(P)*100)
x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
if x == 0:
    result = 99999
else:
    result = 100.*1/x


#Without observed max
ann_max_new, GMST_new = remove_max(ann_max, GMST)
offset_new = []
a, b = np.polyfit(GMST_new, ann_max_new, 1)
for i, each in enumerate(ann_max_new):
    actual = each
    predicted = GMST_new[i]*a +b
    offset_new.append(actual-predicted+(1*a+b)) #adjust for world with 1 deg of warming

#GEV fit without observed max
shape, loc, scale = gev.fit(offset_new) # EVT distribution without observed max
x_val = np.linspace(np.min(offset_new)-.5, np.max(offset_new)+2, 1000)
dist_pdf = gev.pdf(x_val, shape, loc, scale)

offset_full = []
for i, each in enumerate(ann_max):
    actual = each
    predicted = GMST[i]*a +b
    offset_full.append(actual-predicted+(1*a+b))

max_val = np.max(offset_full) #observed max
chance = []
for i, _ in enumerate(dist_pdf):
    P = []
    for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
        P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
    chance.append(sum(P)*100)
x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
if x == 0:
    result_new = 99999
else:
    result_new = 100.*1/x

In [17]:
#Compare ERA5 and JRA55 annual-max data to check for inconsistencies
#Regions that are inconsistent will be masked in subsequent figures
#Runtime 3 sec
era5regannmax_tw=np.loadtxt(maindatadir+'era5output_tw_regions.txt',delimiter=','); 
era5regannmax_t=np.loadtxt(maindatadir+'era5output_t_regions.txt',delimiter=','); 
jra55regannmax_tw=np.loadtxt(maindatadir+'jra55output_tw_regions.txt',delimiter=','); 
jra55regannmax_t=np.loadtxt(maindatadir+'jra55output_t_regions.txt',delimiter=','); 
if domasking==1:
    if maskingvs=='merra2':
        merra2regannmax_tw=np.loadtxt(maindatadir+'merra2output_tw_regions.txt',delimiter=','); 
        merra2regannmax_t=np.loadtxt(maindatadir+'merra2output_t_regions.txt',delimiter=','); 
    

def adjust_obs_1981(ann_max): 
    offset = []
    a, b = np.polyfit(GMST_since1981, ann_max, 1)
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST_since1981[i]*a +b
        offset.append(actual-predicted+(1*a+b)) #1*a+b means adjusted for world with 1deg warming  
    return offset

def year_max_adjust(ann_max):
    x = adjust_obs(ann_max,GMST) 
    year = np.arange(y1, y2+1)
    yr = year[np.where(x==np.amax(x))[0]][0]
    return yr

def year_max_adjust_1981(ann_max):
    x = adjust_obs_1981(ann_max) 
    year = np.arange(y1+20, y2+1) #1981-end
    yr = year[np.where(x==np.amax(x))[0]][0]
    return yr

def year_max(ann_max):
    x = ann_max
    year = np.arange(y1, y2)
    yr = year[np.where(x==np.amax(x))[0]][0]
    return yr

def year_max_multiple_adjust(ann_max, how_many):
    x = adjust_obs(ann_max,GMST) #detrend timeseries
    year = np.arange(y1, y2+1)
    yr = []
    for each in np.sort(x)[-how_many:]:
        yr.append(year[np.where(x==each)[0]][0])
    return yr

def year_max_multiple_adjust_1981(ann_max, how_many):
    x = adjust_obs_1981(ann_max) #detrend timeseries
    year = np.arange(y1+20, y2+1) #1981-end
    yr = []
    for each in np.sort(x)[-how_many:]:
        yr.append(year[np.where(x==each)[0]][0])
    return yr


#zeros -- regions to be masked due to inconsistency between datasets
if domasking==1:
    #Tw
    mylist = []
    for reg in range(numregs):
        if maskingvs=='merra2':
            ann_max=merra2regannmax_tw[reg][20:];how_many=5;
            merra2_yr=year_max_multiple_adjust_1981(ann_max, how_many); #top few years in this region selon MERRA2

            ann_max=era5regannmax_tw[reg][20:];
            era_yr=year_max_adjust_1981(ann_max); #top year for ERA5

            if era_yr in merra2_yr:
                mylist.append(1)
            else:
                mylist.append(0)
                
        elif maskingvs=='jra55':
            ann_max=jra55regannmax_tw[reg];how_many=5;
            jra55_yr=year_max_multiple_adjust(ann_max, how_many); #top few years in this region selon JRA55

            ann_max=era5regannmax_tw[reg];
            era_yr=year_max_adjust(ann_max); #top year for ERA5

            if era_yr in jra55_yr:
                mylist.append(1)
            else:
                mylist.append(0)

    mask_regs_tw = mylist
    
    #T
    mylist = []
    for reg in range(numregs):
        if maskingvs=='merra2':
            ann_max=merra2regannmax_t[reg][20:];how_many=5;
            merra2_yr=year_max_multiple_adjust_1981(ann_max, how_many); #top few years in this region selon MERRA2

            ann_max=era5regannmax_t[reg][20:];
            era_yr=year_max_adjust_1981(ann_max); #top year for ERA5

            if era_yr in merra2_yr:
                mylist.append(1)
            else:
                mylist.append(0)
                
        elif maskingvs=='jra55':
            ann_max=jra55regannmax_t[reg];how_many=5;
            jra55_yr=year_max_multiple_adjust(ann_max, how_many); #top few years in this region selon JRA55

            ann_max=era5regannmax_t[reg];
            era_yr=year_max_adjust(ann_max); #top year for ERA5

            if era_yr in jra55_yr:
                mylist.append(1)
            else:
                mylist.append(0)

    mask_regs_t = mylist
else:
    mask_regs_tw=np.ones((numregs,));
    mask_regs_t=np.ones((numregs,));

np.savetxt(maindatadir+"mask_regs_tw_"+maskingvs+".csv", np.array(mask_regs_tw), delimiter=",",fmt='%10.0f')
np.savetxt(maindatadir+"mask_regs_t_"+maskingvs+".csv", np.array(mask_regs_t), delimiter=",",fmt='%10.0f')

In [87]:
#Compute difference between record and 1-in-100 event using GMST-adjusted data
#Uses ERA5/JRA55 mean
#Runtime 15 sec
era5regannmax_adj_tw=np.loadtxt(maindatadir+'era5output_tw_adj_regions.txt',delimiter=',');
#jra55regannmax_adj_tw=np.loadtxt(maindatadir+'jra55output_tw_adj_regions.txt',delimiter=',');
hundred_rec_diff=np.zeros((numregs,));
for reg in range(numregs):
    era5arr=era5regannmax_adj_tw[reg,:];
    #jra55arr=jra55regannmax_adj_tw[reg,:];
    #myarr=(era5arr+jra55arr)/2;
    myarr=era5arr;
    myrec=np.max(myarr);
    
    oneinhundredval=one_in_hundred(myarr);
    hundred_rec_diff[reg]=oneinhundredval-myrec; #i.e. 1-in-100 value is this many degrees higher

In [27]:
#For ERA5: set up regional data, implement regional mask if desired, and do GEV fit
#Note that this loop ingests raw data and GMST-adjusts it -- don't feed it already GMST-adjusted data!!
#Cell runtime: 4 min for all loops
if domainprep==1:
    regs_in_ant = np.arange(170, 191);numregs_ant=21; #ordinates of the 21 Antarctica regions to remove
    numregs_noant=numregs-numregs_ant;

    #Preliminarily load region data
    region_fp = maindatadir+'region_fx-WRAF05-v4-1_WRAF_All-Hist_est1_v4-1_4-1-0_000000-000000.nc'
    region_ds = xr.open_mfdataset(region_fp, parallel=True)
    lats = region_ds.lat
    lons = region_ds.lon
    
    orignumlats=np.shape(region_ds.region)[0];orignumlons=orignumlats*2;
    
    #Reduce lat and lon resolution each by a factor of 2 (still very high res) to avoid killing the kernel
    if ~('coarserregs' in globals()):
        coarserregs=np.round(regrid(np.array(region_ds.region.values),int(orignumlats/2),int(orignumlons/2)));
    

    for loop in range(3,4): #default is 0,3
        if loop==0:
            var='tw';fnamepart='tw_era5';regannmax=era5regannmax_tw;mask_regs=mask_regs_tw;
        elif loop==1:
            var='t';fnamepart='t_era5';regannmax=era5regannmax_t;mask_regs=mask_regs_t;
        elif loop==2: #Mean of datasets
            var='twdatasetmean';fnamepart='tw_datasetmean';regannmax=(era5regannmax_tw+jra55regannmax_tw)/2;mask_regs=mask_regs_tw;
        elif loop==3: #JRA55 Tw
            var='tw_jra55';fnamepart='tw_jra55';regannmax=jra55regannmax_tw;mask_regs=mask_regs_tw;
            
        regords_noant = [] #region ordinates
        vals_actual_noant = [] #actual values
        vals_retper_noant = [] #return periods
        vals_retper_noant_withtopevents = [] #return periods with top events included
        vals_abs_noant = [] #difference between observed max and statistical max
        for each in np.arange(numregs):
            regords_noant.append(each)
            vals_actual_noant.append(np.max(regannmax[each,:]))
            vals_retper_noant.append(obs_max_ret_per_removeevent(each,var)) #Return periods
            vals_retper_noant_withtopevents.append(obs_max_ret_per(each,var)) #Return periods with top events included
            vals_abs_noant.append(offset_higher_removeevent(each,var)) #Highest values (C)

        #vals_retper_noant = [np.nan if x == 99999 else x for x in vals_retper_noant]

        # Convert vals to regional data for mapping
        region_ord=coarserregs; #just to initialize
        region_actual=coarserregs;
        region_abs = coarserregs;
        region_retper = coarserregs;
        region_retper_withtopevents = coarserregs;
        for r in range(numregs):
            region_ord = np.where(coarserregs != r, region_ord, regords_noant[r]); #condition, val if true, val if false
            region_actual = np.where(coarserregs != r, region_actual, vals_actual_noant[r])
            region_retper = np.where(coarserregs != r, region_retper, vals_retper_noant[r])
            region_retper_withtopevents = np.where(coarserregs != r, region_retper_withtopevents, vals_retper_noant_withtopevents[r])
            region_abs = np.where(coarserregs != r, region_abs, vals_abs_noant[r])
            #region_abs = region_abs.where(coarserregs != r, vals_abs_noant[r])


        # Zeros in mask arrays-- regions that will be shaded dark grey in the final plot
        # Antarctica not included here as it is assumed to be excluded

        #For comparison -- mask from Thompson et al. 2023
        reg_mask_t = [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
                    0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 
                    0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 
                    1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 
                    0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1];

        #reg_mask=np.concatenate((np.transpose(mask_regs[:170]),np.transpose(mask_regs[191:238])));
        reg_mask=mask_regs;
            #good regions are 1, bad regions are 0
        reg_mask = [np.nan if x == 1 else x for x in reg_mask] 
            #good regions are nan (i.e. will not be covered up), bad regions are 0
        #region_mask = np.array(region_ds.region); #initialize
        region_mask = coarserregs;

        #NEW
        for region in range(numregs):
            region_mask[region_mask==region]=reg_mask[region];



        #For reference   
        regcenterpoints=np.loadtxt(maindatadir+'region_centerpoints.csv',delimiter=',');
        regcenterlats_tmp=regcenterpoints[:,0].tolist();
        regcenterlons_tmp=regcenterpoints[:,1].tolist();
        regcenterlats_noant=np.concatenate((np.transpose(regcenterlats_tmp[:170]),np.transpose(regcenterlats_tmp[191:238])));
        regcenterlons_noant=np.concatenate((np.transpose(regcenterlons_tmp[:170]),np.transpose(regcenterlons_tmp[191:238])));


        np.savetxt(maindatadir+"currentrecordreturnperiods_"+fnamepart+".csv", np.array(vals_retper_noant), 
                   delimiter=",",fmt='%10.2f')
        np.savetxt(maindatadir+"currentrecordreturnperiods_withtopevents_"+fnamepart+".csv", 
                   np.array(vals_retper_noant_withtopevents), delimiter=",",fmt='%10.2f')

        if loop==0:
            region_ord_tw=region_ord;region_actual_tw=region_actual;region_retper_tw=region_retper;
            region_abs_tw=region_abs;region_mask_tw=region_mask;
            vals_actual_noant_tw=vals_actual_noant;
            vals_retper_noant_tw=vals_retper_noant;
            vals_retper_noant_withtopevents_tw=vals_retper_noant_withtopevents;
            vals_abs_noant_tw=vals_abs_noant;
        elif loop==1:
            region_ord_t=region_ord;region_actual_t=region_actual;region_retper_t=region_retper;
            region_abs_t=region_abs;region_mask_t=region_mask;
            vals_actual_noant_t=vals_actual_noant;vals_retper_noant_t=vals_retper_noant;
            vals_abs_noant_t=vals_abs_noant;
        elif loop==2:
            region_ord_tw_datasetmean=region_ord;region_actual_tw_datasetmean=region_actual;
            region_retper_tw_datasetmean=region_retper;
            region_abs_tw_datasetmean=region_abs;region_mask_tw_datasetmean=region_mask;
            vals_actual_noant_tw_datasetmean=vals_actual_noant;vals_retper_noant_tw_datasetmean=vals_retper_noant;
            vals_abs_noant_tw_datasetmean=vals_abs_noant;
            
        #del region_ord;del region_actual;del region_retper;del region_abs;del region_mask;

        #plt.imshow(region_mask);plt.colorbar()

del region_ds

In [14]:
#Map of actual regional Tw annual maxes (from ERA5) -- 1 min
dothis=0;
if dothis==1:
    fig, axs = plt.subplots(2,1,figsize=(10., 7.), dpi=80, num=None, subplot_kw={'projection': ccrs.PlateCarree()})
    c = axs[0].contourf(lons,lats,region_actual_tw,11,transform=ccrs.PlateCarree(),
                        cmap=mpl_cm.get_cmap('bwr'), vmin=0, vmax=30) # or try 'bwr''coolwarm'
    cbar = plt.colorbar(c, shrink=0.7, ax=axs[0])
    b = axs[0].contourf(lons,lats,region_mask_tw,11,transform=ccrs.PlateCarree(), colors='darkgrey', vmin=-1, vmax=1)
    plt.tight_layout()
    
    
#Map of region ordinates -- 1 min
#fig, axs = plt.subplots(2,1,figsize=(10., 7.), dpi=80, num=None, subplot_kw={'projection': ccrs.PlateCarree()})
#c = axs[0].contourf(lons,lats,region_ord,11,transform=ccrs.PlateCarree(),
#                    cmap=mpl_cm.get_cmap('jet'), vmin=0, vmax=numregs) # or try 'bwr''coolwarm'
#cbar = plt.colorbar(c, shrink=0.7, ax=axs[0])
#b = axs[0].contourf(lons,lats,region_mask,11,transform=ccrs.PlateCarree(), colors='darkgrey', vmin=-1, vmax=1)
#plt.tight_layout()

In [15]:
#Actually make analogue to T23 Figure 2
#Cell runtime: 1 min per variable

if maket23fig2==1:
    var='tw'; #can be 'tw' or 't'

    if var=='tw':
        region_abs=region_abs_tw;region_retper=region_retper_tw;region_mask=region_mask_tw;var_title='Tw';
    else:
        region_abs=region_abs_t;region_retper=region_retper_t;region_mask=region_mask_t;var_title='T';

    fig, axs = plt.subplots(2, 1, figsize=(10., 7.), dpi=80, num=None, subplot_kw={'projection': ccrs.PlateCarree()})
    # Statistical maximum difference plot
    c = axs[0].contourf(lons,lats,region_abs,11,transform=ccrs.PlateCarree(),
                        cmap=mpl_cm.get_cmap('bwr'), vmin=-3.5, vmax=3.5) # or try 'bwr''coolwarm'
    cbar = plt.colorbar(c, shrink=0.7, ax=axs[0])
    b = axs[0].contourf(lons,lats,region_mask,11,transform=ccrs.PlateCarree(), colors='darkgrey', vmin=-1, vmax=1)
    cbar.set_label(u"\u2103", fontsize=12)
    cbar.outline.set_linewidth(0.5)
    #cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(), ha='right')
    cbar.ax.tick_params(labelsize=12,pad=40)
    axs[0].set_title('Statistical maximum ('+var_title+') minus current record')
    axs[0].text(-190, 95, 'a')
    # Current return period plot
    e = axs[1].contourf(lons,lats,region_abs,11,transform=ccrs.PlateCarree(),
                        colors='r',
                        vmin=-10, vmax=30)
    levels=np.logspace(np.log10(1),np.log10(10000),5)
    d = axs[1].contourf(lons,lats,region_retper,11,transform=ccrs.PlateCarree(),
                        cmap=mpl_cm.get_cmap('Blues'),
                        levels=levels, locator=ticker.LogLocator(base=10))
    b = axs[1].contourf(lons,lats,region_mask,11,transform=ccrs.PlateCarree(), colors='darkgrey', vmin=-1, vmax=1)

    axs[1].set_title('Current record return period ('+var_title+')')
    axs[1].text(-190, 95, 'b')
    cbar = plt.colorbar(d, shrink=0.7, ax = axs[1], ticks=ticker.LogLocator(base=10))
    cbar.set_label('Years', fontsize=12)
    cbar.outline.set_linewidth(0.5)
    cbar.ax.tick_params(labelsize=12,width=0.5)

    axs[1].text(-170, -35, 'Exceptional', fontsize = 10, 
             bbox = dict(facecolor = 'red', alpha = 0.5))
    axs[1].text(-170, -50, 'Inconsistent data', fontsize = 10, 
             bbox = dict(facecolor = 'darkgrey', alpha = 0.5))

    for axes in axs:
        axes.coastlines()
        axes.set_ylim([-60, 90])
        #axes.outline_patch.set_linewidth(1)

    plt.tight_layout()
    plt.savefig('replication_fig2_'+var+'.png', dpi=300)
    #plt.savefig('fig2_final.pdf')
    #plt.close()

In [ ]:
#Make T23 Figure 3
#Cell runtime: 10 sec

#Freshly redefine reg_mask
#reg_mask=np.concatenate((np.transpose(mask_regs[:170]),np.transpose(mask_regs[191:238])));
#reg_mask = [np.nan if x == 1 else x for x in reg_mask] 

if maket23fig3==1:
    for loop in range(0,2):
        if loop==0:
            var='tw';reg_mask=mask_regs_tw;
            vals_retper_noant=vals_retper_noant_tw;vals_abs_noant=vals_abs_noant_tw;
            blabelpos=1.55;
        elif loop==1:
            var='t';reg_mask=mask_regs_t;
            vals_retper_noant=vals_retper_noant_t;vals_abs_noant=vals_abs_noant_t;
            blabelpos=4.55;


        reg_mask = [np.nan if x == 1 else x for x in reg_mask] 


        # Remove the Antarctic regions, then apply the masking
        reg_retper = vals_retper_noant.copy()
        reg_abs = vals_abs_noant.copy()

        for each in regs_in_ant[::-1]:
            reg_retper.pop(each)
            reg_abs.pop(each)
            reg_mask.pop(each)

        retper_mask = []
        abs_mask = []
        for i, each in enumerate(reg_mask):
            if np.isnan(each):
                retper_mask.append(reg_retper[i])
                abs_mask.append(reg_abs[i])
            else: pass

        ## Scatter of two measures - where nan
        def obs_max_year(reg,var):
            ' Returns the highest event in terms of return period, yrs '
            ann_max = load_annmax(reg,var)
            max = np.where(ann_max == np.max(ann_max))[0][0]
            return max+y1 # index -> year

        vals_wherenan = []
        year_wherenan = []
        for i, each in enumerate(reg_retper):
            if np.isnan(reg_mask[i]):
                if np.isnan(each):
                    vals_wherenan.append(reg_abs[i])
                    year_wherenan.append(obs_max_year(i,var))


        ## regions' population data, from excel sheet
        pop2020 = [35058, 9362.0, 36778.0, 4244.0, 7085, 155.0, 7784.0, 123699, 3768962, 3838695, 1275069, 1463330.0, 
               280330.0, 14180861.0, 39153.0, 7285067, 2248624, 6525, 573490.0, 11999838, 2697617, 8547714, 24082918.0,
               11842203, 39273866.0, 40107493, 8384732, 24665718, 12264077, 63378369.0, 35493739, 33887421, 1776601, 
               443288, 4961768.0, 1375779, 6042843, 31150375, 11773314, 619124, 1984691, 10275139, 39445384, 80088461, 
               4127100, 4561119, 2973924, 30302763, 495720, 6688702, 29372864, 1777633, 32093533, 173174, 3626562, 
               35863592, 686558, 185412, 1058910, 1833886, 5124811, 9067462, 4789973, 28279744, 86327318, 1185522, 
               75910653, 13776194, 14810497, 5318217, 20417405, 29934246, 7901607, 32427030, 2057012, 17175842, 
               25703520, 64430782, 498206, 19633883, 73703476.0, 114829191, 228727, 13651192.0, 22070496.0, 4668836, 
               6345789, 17137553, 17125552.0, 28038064, 18433425.0, 11323533, 12411156, 35074065.0, 35005949.0, 
               17973787.0, 25106274, 25749291.0, 2580356, 2124047, 12522729, 39674125, 3300780, 884918, 40451661, 
               22175302, 23963980, 19123720, 302242.0, 1516770, 4522402, 9248667, 182013.0, 513, 3062310, 1289081.0, 
               2773401.0, 1054825, 281, 114.0, 104012, 9724, 293647, 132290, 17011, 356977, 2011401, 1432453, 74370541,
               5351986, 27571533.0, 53411661, 14602262, 12217481, 1233655, 4359458, 3241555.0, 3239493.0, 5006886.0, 
               38755552, 197725186, 10665078, 7519179, 1034334, 31269113, 4808374, 11947485, 13950923, 67500374, 
               250297823.0, 704816, 1048482, 115213313, 75085565, 176968847, 267801195, 142034915, 144861962, 21875, 
               46092, 601818, 48198, 6191, 1245302, 7596027, 42121, 49716, 316674, 395350, 2590528, 0.0, 0.0, 0.0, 0.0,
               0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47199454, 2288772, 
               22792175, 1392269, 3761272, 7087663, 75580294.0, 9126748, 10695544, 110276787.0, 65017756, 51612076.0, 
               64729722, 68988873, 34947609, 153658397, 427316337, 173377780, 258059259, 279816308, 44303465, 110458041,
               69831788, 114120493, 203805030, 12323013, 17031880, 2039.0, 0.0, 872.0, 8044559.0, 16474411, 73425997, 
               21716337, 10139072, 84906918, 14565540, 91121993.0, 600857, 2097275, 349775, 45466417.0, 17321153.0, 
               41271482, 153926924, 7491016];


        #Also read in region center points, so regions can be tracked after masking, etc
        regcenterpoints=np.loadtxt(maindatadir+'region_centerpoints.csv',delimiter=',');
        regcenterlats=regcenterpoints[:,0].tolist();
        regcenterlons=regcenterpoints[:,1].tolist();

        origords=[*range(0,numregs)];


        for each in regs_in_ant[::-1]:
            pop2020.pop(each)
            regcenterlats.pop(each)
            regcenterlons.pop(each)
            origords.pop(each)

        pop_mask = [];centerlats_mask = [];centerlons_mask = [];origords_mask=[];
        for i, each in enumerate(reg_mask):
            if np.isnan(each):
                pop_mask.append(pop2020[i])
                centerlats_mask.append(regcenterlats[i])
                centerlons_mask.append(regcenterlons[i])
                origords_mask.append(origords[i])
            else: pass


        #Now, we're ready to do the plotting
        fig, axs = plt.subplots(1, 2, figsize=(10., 4.), dpi=80)
        axs[0].plot(retper_mask, pop_mask, '+', label='Regional records')
        axs[0].set_xlabel('Return Period of current record (Years)', fontsize=12)
        axs[0].set_ylabel('Population in 2020', fontsize=12)
        axs[0].set_xscale('log')
        axs[0].set_yscale('log')
        axs[0].grid()
        #axs[0].plt.arrow(10**6, 10**3, 10**5, 10**
        axs[0].text(17, 8E8, 'a')
        axs[0].legend(loc='lower right')
        for i in range(len(vals_wherenan)): vals_wherenan[i] = -vals_wherenan[i]

        axs[1].plot(year_wherenan, vals_wherenan, '+', label='Regional records')
        #if var=='t':
            #axs[1].plot([2021, 2021], np.sort(vals_wherenan)[-2:], 'r+', label='Western North America heatwave')
        axs[1].set_xlim(y1-1,y2+1);
        axs[1].set_xlabel('Year', fontsize=12)
        axs[1].set_ylabel('How far beyond statistical maximum, '+u"\u2103", fontsize=12)
        axs[1].grid()
        axs[1].text(y1-2, blabelpos, 'b')
        axs[1].legend(loc='upper left')

        plt.tight_layout()
        plt.savefig('fig3_final_'+var+'.png', dpi=300)
        #plt.savefig('fig3_final.pdf')
        #plt.close()

In [15]:
#Functions supporting model analysis
def remove_max_model(ann_max, GMST):
    ' Remove the max value from region, and corresponding GMST value '
    ann_max_new = []
    GMST_new = []
    for ens in np.arange(np.shape(ann_max)[0]):
        ann_max_new.append(np.delete(ann_max[ens], np.abs(ann_max[ens] - np.max(ann_max[ens])).argmin()))
        GMST_new.append(np.delete(GMST, np.abs(ann_max[ens] - np.max(ann_max[ens])).argmin()))
    return ann_max_new, GMST_new

def calc_fit(data, GMST):
    # data & GMST must be same shape
    A, B = np.shape(data)
    offset = []
    model = np.reshape(data, [A*B])
    gmst = np.reshape(GMST, [A*B])
    a, b = np.polyfit(gmst, model, 1)
    return a, b

def detrend(data, gm, a, b, enssz):
    A, B = np.shape(data)
    offset_data = []
    actual = data
    predicted = np.reshape(gm*a+b, [enssz, numyrs])
    offset = actual-predicted+(1*a+b) #for a GMST of 1C
    return offset

#Where the GEVs are fit!
def return_period(GEV_data, extreme):
    shape, loc, scale = gev.fit(GEV_data) # EVT distribution
    x_val = np.linspace(np.min(GEV_data)-.5, np.max(GEV_data)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    chance = []
    for i, _ in enumerate(dist_pdf):
        P = []
        for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
            P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
        chance.append(sum(P)*100)
    x = chance[np.abs(x_val - extreme).argmin()] # chance of observed max
    if x == 0:
        result = np.nan
    else:
        result = 100.*1/x
    return result

In [ ]:
#Adjust ERA5, MERRA2, and JRA55 annual-max data to a GMST of 1C
#This prepares the way for daily data to be adjusted (which is done in Matlab)
#Runtime 3 sec
for loop in range(0,2):
    if loop==0:
        var='tw';
    elif loop==1:
        var='t';
        
    era5annmax=np.loadtxt(maindatadir+'era5output_'+var+'_regions.txt',delimiter=',').reshape((numregs,numyrs));
    merra2annmax=np.loadtxt(maindatadir+'merra2output_'+var+'_regions.txt',delimiter=',').reshape((numregs,numyrs));
    jra55annmax=np.loadtxt(maindatadir+'jra55output_'+var+'_regions.txt',delimiter=',').reshape((numregs,numyrs));
    era5adjfactor=np.zeros((numregs,numyrs));
    merra2adjfactor=np.zeros((numregs,numyrs));
    jra55adjfactor=np.zeros((numregs,numyrs));
    for reg in range(numregs):
        a,b=np.polyfit(GMST,era5annmax[reg,:],1); # ERA5 annual max for this region versus GMST    
        #Detrend
        fittogmst_era5 = np.asarray(GMST)*a+b;
        data_era5_adj = era5annmax[reg,:]-fittogmst_era5+(1*a+b) #for a GMST of 1C
        era5adjfactor[reg,:]=data_era5_adj-era5annmax[reg,:]; #for transferring to Matlab to then adjust daily values


        a,b=np.polyfit(GMST[20:],merra2annmax[reg,20:],1); # MERRA2 annual max for this region versus GMST   
        #Detrend
        fittogmst_merra2 = np.asarray(GMST[20:])*a+b;
        data_merra2_adj = merra2annmax[reg,20:]-fittogmst_merra2+(1*a+b) #for a GMST of 1C
        merra2adjfactor[reg,20:]=data_merra2_adj-merra2annmax[reg,20:]; #for transferring to Matlab to then adjust daily values
        
        
        a,b=np.polyfit(GMST,jra55annmax[reg,:],1); # JRA55 annual max for this region versus GMST   
        #Detrend
        fittogmst_jra55 = np.asarray(GMST)*a+b;
        data_jra55_adj = jra55annmax[reg,:]-fittogmst_jra55+(1*a+b) #for a GMST of 1C
        jra55adjfactor[reg,:]=data_jra55_adj-jra55annmax[reg,:]; #for transferring to Matlab to then adjust daily values

        
    era5adjfactor[era5adjfactor == 0] =np.nan;
    merra2adjfactor[merra2adjfactor == 0] =np.nan;
    jra55adjfactor[jra55adjfactor == 0] =np.nan;

    np.savetxt(maindatadir+"era5adjfactors_"+var+".csv",era5adjfactor,delimiter=",",fmt='%10.2f');
    np.savetxt(maindatadir+"merra2adjfactors_"+var+".csv",merra2adjfactor,delimiter=",",fmt='%10.2f');
    np.savetxt(maindatadir+"jra55adjfactors_"+var+".csv",jra55adjfactor,delimiter=",",fmt='%10.2f');

In [86]:
#Adjust model annual-max data to a GMST of 1C
#This prepares the way for daily data to be adjusted (which is done in Matlab)
#Runtime 5 sec
for loop in range(0,1):
        
    myrs=60; 
        
    canesm5annmax=np.loadtxt(maindatadir+'canesm5output_regions.txt',delimiter=',').reshape((canesm5_enssz,numregs,numyrs));
    miroc6annmax=np.loadtxt(maindatadir+'miroc6output_regions.txt',delimiter=',').reshape((miroc6_enssz,numregs,numyrs));
    mpigeannmax=np.loadtxt(maindatadir+'mpigeoutput_regions.txt',delimiter=',').reshape((mpige_enssz,numregs,numyrs));
    canesm5adjfactor=np.zeros((canesm5_enssz,numregs,numyrs));
    miroc6adjfactor=np.zeros((miroc6_enssz,numregs,numyrs));
    mpigeadjfactor=np.zeros((mpige_enssz,numregs,numyrs));
    for reg in range(numregs):
        for ensmem in range(0,canesm5_enssz):
            a,b=np.polyfit(GMST,canesm5annmax[ensmem,reg,:],1); # CanESM5 annual max for this region versus GMST    
            #Detrend
            fittogmst_canesm5 = np.asarray(GMST)*a+b;
            data_canesm5_adj = canesm5annmax[ensmem,reg,:]-fittogmst_canesm5+(1*a+b) #for a GMST of 1C
            canesm5adjfactor[ensmem,reg,:]=data_canesm5_adj-canesm5annmax[ensmem,reg,:]; #for xfer to Matlab

        for ensmem in range(0,miroc6_enssz):
            a,b=np.polyfit(GMST[20:],miroc6annmax[ensmem,reg,20:],1); # MIROC6 annual max for this region versus GMST   
            #Detrend
            fittogmst_miroc6 = np.asarray(GMST[20:])*a+b;
            data_miroc6_adj = miroc6annmax[ensmem,reg,20:]-fittogmst_miroc6+(1*a+b) #for a GMST of 1C
            miroc6adjfactor[ensmem,reg,20:]=data_miroc6_adj-miroc6annmax[ensmem,reg,20:]; #for xfer to Matlab

        for ensmem in range(0,mpige_enssz):
            a,b=np.polyfit(GMST,mpigeannmax[ensmem,reg,:],1); # MPI-GE annual max for this region versus GMST   
            #Detrend
            fittogmst_mpige = np.asarray(GMST)*a+b;
            data_mpige_adj = mpigeannmax[ensmem,reg,:]-fittogmst_mpige+(1*a+b) #for a GMST of 1C
            mpigeadjfactor[ensmem,reg,:]=data_mpige_adj-mpigeannmax[ensmem,reg,:]; #for xfer to Matlab

        
    canesm5adjfactor[canesm5adjfactor == 0] =np.nan;
    miroc6adjfactor[miroc6adjfactor == 0] =np.nan;
    mpigeadjfactor[mpigeadjfactor == 0] =np.nan;

    np.savetxt(maindatadir+"canesm5adjfactors_tw.csv",canesm5adjfactor.flatten(order='F'),delimiter=",",fmt='%10.2f');
    np.savetxt(maindatadir+"miroc6adjfactors_tw.csv",miroc6adjfactor.flatten(order='F'),delimiter=",",fmt='%10.2f');
    np.savetxt(maindatadir+"mpigeadjfactors_tw.csv",mpigeadjfactor.flatten(order='F'),delimiter=",",fmt='%10.2f');

In [169]:
#Troubleshooting (of the cell below) for a selected region
troubleshoot=0;
if troubleshoot==1:
    reg=160;
    ann_max = modelannmax[:,reg,:]; #2D array, with data for each ens mem and year
    a, b = calc_fit(ann_max, np.tile(GMST, enssz)) # annual max for this region versus GMST
    offset = detrend(ann_max, np.tile(GMST, enssz), a, b,enssz)
    ann_max_new_detrend, GMST_new_detrend = remove_max_model(offset, GMST)

    max_3yr=np.zeros((enssz,numblocks)); #2D array, with data for each ens mem and 3-year block maxima
    for ens in np.arange(enssz):
        for i in range(blsz-1,numyrs,blsz):
            max_3yr[ens,int((i+1)/blsz-1)]=np.max(ann_max[ens,i-(blsz-1):i+1]);

    max_1yr = ann_max_new_detrend #detrended annual maxes
    model_extremes = np.max(offset, axis=1) #physically modeled extremes
    ret_per_1yr = [];ret_per_3yr = [];

    #Fit GEV (based on annual maxima)
    for ens in np.arange(enssz):
        ret_per_1yr.append(return_period(max_1yr[ens], model_extremes[ens]))
    my_impossible_count=(sum(np.isnan(x) for x in ret_per_1yr))

    #Fit GEV using 5-year block maxima instead
    blockmax_3yr=np.zeros((enssz,numblocks));
    for ens in np.arange(enssz):
        for i in range(blsz-1,numyrs,blsz):
            blockmax_3yr[ens,int((i+1)/blsz-1)]=np.max(max_1yr[ens][i-(blsz-1):i+1]);

        ret_per_3yr.append(return_period(blockmax_3yr[ens], model_extremes[ens]))
    my_impossible_count_3yr=(sum(np.isnan(x) for x in ret_per_3yr))


    #Place in a separate cell for serious troubleshooting
    ens=0
    GEV_data=blockmax_3yr[ens]
    extreme=model_extremes[ens]

    shape, loc, scale = gev.fit(GEV_data) # EVT distribution
    x_val = np.linspace(np.min(GEV_data)-.5, np.max(GEV_data)+2, 1000)
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    chance = []
    for i, _ in enumerate(dist_pdf):
        P = []
        for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
            P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
        chance.append(sum(P)*100)
    x = chance[np.abs(x_val - extreme).argmin()] # chance of observed max
    if x == 0:
        result = np.nan
    else:
        result = 100.*1/x

    np.round(result)

    #plt.plot(ret_per_3yr)


    #Evaluate GEV and return periods across full ensemble
    #model_forGEV = np.sort(np.reshape(ann_max.data, [numyrs*enssz,]))[:-1] #ann maxes including overall maximum
    #model_extreme = np.max(np.reshape(ann_max.data, [numyrs*enssz,])); #maximum
    #my_ret_per_full=np.round(return_period(model_forGEV, model_extreme))

    #model_forGEV_3yr = np.sort(np.reshape(max_3yr.data, [numblocks*enssz,]))[:-1] #everything but maximum
    #model_extreme_3yr = np.max(np.reshape(max_3yr.data, [numblocks*enssz,])); #maximum
    #my_ret_per_full_3yr=np.round(return_period(model_forGEV_3yr, model_extreme_3yr))

In [28]:
whos

Variable                          Type              Data/Info
-------------------------------------------------------------
GMST                              list              n=63
GMST_new                          ndarray           62: 62 elems, type `float64`, 496 bytes
GMST_since1981                    list              n=43
GMST_yr                           ndarray           63: 63 elems, type `int64`, 504 bytes
LogNorm                           type              <class 'matplotlib.colors.LogNorm'>
P                                 list              n=0
RegularGridInterpolator           type              <class 'scipy.interpolate<...>RegularGridInterpolator'>
a                                 float64           0.0
abs_max                           function          <function abs_max at 0x17587e660>
actual                            float64           19.024
add_season_membership             function          <function add_season_membership at 0x11eb8c400>
adjust_obs                 

In [133]:
#Computes percent of model ensemble members showing record events outside the GEV fit
#Runtime: about 45 min for CanESM5, 10 min for MIROC6, 25 min for MPI
#Once run, results can be reloaded
if rerunmodelcomp==1:
    for model in range(0,3): #default is 0,3
        if model==0:
            enssz=canesm5_enssz;mname='canesm5';
            impossible_pct_canesm5=[];ret_per_full_canesm5=[];
            impossible_pct_3yr_canesm5=[];ret_per_full_3yr_canesm5=[];
        elif model==1:
            enssz=miroc6_enssz;mname='miroc6';
            impossible_pct_miroc6=[];ret_per_full_miroc6=[];
            impossible_pct_3yr_miroc6=[];ret_per_full_3yr_miroc6=[];
        elif model==2:
            enssz=mpige_enssz;mname='mpige';
            impossible_pct_mpige=[];ret_per_full_mpige=[];
            impossible_pct_3yr_mpige=[];ret_per_full_3yr_mpige=[];

        impossible_count=[];ret_per_full=[];
        impossible_count_3yr=[];ret_per_full_3yr=[];
        blsz=3; #alternative (to annual) block size, in years
        numblocks=int(numyrs/blsz);

        modelannmax=np.loadtxt(maindatadir+mname+'output_regions.txt',delimiter=',').reshape((enssz,numregs,numyrs));
        
        for reg in range(numregs):
            ann_max = modelannmax[:,reg,:]; #2D array, with data for each ens mem and year
            a, b = calc_fit(ann_max, np.tile(GMST, enssz)) # annual max for this region versus GMST
            offset = detrend(ann_max, np.tile(GMST, enssz), a, b,enssz)
            ann_max_new_detrend, GMST_new_detrend = remove_max_model(offset, GMST)
            
            max_3yr=np.zeros((enssz,numblocks)); #2D array, with data for each ens mem and 3-year block maxima
            for ens in np.arange(enssz):
                for i in range(blsz-1,numyrs,blsz):
                    max_3yr[ens,int((i+1)/blsz-1)]=np.max(ann_max[ens,i-(blsz-1):i+1]);
            
            model_forGEV = ann_max_new_detrend #detrended annual maxes
            model_extremes = np.max(offset, axis=1) #physically modeled extremes
            ret_per_1yr = [];ret_per_3yr = [];
            
            #Fit GEV (based on annual maxima)
            for ens in np.arange(enssz):
                ret_per_1yr.append(return_period(model_forGEV[ens], model_extremes[ens]))
            impossible_count.append(sum(np.isnan(x) for x in ret_per_1yr))
            
            #Fit GEV using 5-year block maxima instead
            blockmax=np.zeros((enssz,numblocks));
            for ens in np.arange(enssz):
                for i in range(blsz-1,numyrs,blsz):
                    blockmax[ens,int((i+1)/blsz-1)]=np.max(model_forGEV[ens][i-(blsz-1):i+1]);
                    
                ret_per_3yr.append(return_period(blockmax[ens], model_extremes[ens]))
            impossible_count_3yr.append(sum(np.isnan(x) for x in ret_per_3yr))
            
            
            #Evaluate GEV and return periods across full ensemble
            model_forGEV = np.sort(np.reshape(ann_max.data, [numyrs*enssz,]))[:-1] #ann maxes including overall maximum
            model_extreme = np.max(np.reshape(ann_max.data, [numyrs*enssz,])); #maximum
            ret_per_full.append(np.round(return_period(model_forGEV, model_extreme)))
            #print(reg, impossible_count[-1], ret_per_full[-1])
            
            model_forGEV_3yr = np.sort(np.reshape(max_3yr.data, [numblocks*enssz,]))[:-1] #everything but maximum
            model_extreme_3yr = np.max(np.reshape(max_3yr.data, [numblocks*enssz,])); #maximum
            ret_per_full_3yr.append(np.round(return_period(model_forGEV_3yr, model_extreme_3yr)))

            
        # For all regions, adjust number to be a percentage of ens members outside GEV fit
        multiplyby=100/enssz;
        impossible_pct=[x*multiplyby for x in impossible_count];
        impossible_pct_3yr=[x*multiplyby for x in impossible_count_3yr];

        if model==0:
            impossible_pct_canesm5=impossible_pct;ret_per_full_canesm5=ret_per_full;
            impossible_pct_3yr_canesm5=impossible_pct_3yr;ret_per_full_3yr_canesm5=ret_per_full_3yr;
            
            np.savetxt(maindatadir+"impossiblepct_canesm5.csv",np.array(impossible_pct_canesm5),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_canesm5.csv",np.array(ret_per_full_canesm5),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"impossiblepct_3yr_canesm5.csv",np.array(impossible_pct_3yr_canesm5),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_3yr_canesm5.csv",np.array(ret_per_full_3yr_canesm5),delimiter=",",fmt='%10.2f')
        elif model==1:
            impossible_pct_miroc6=impossible_pct;ret_per_full_miroc6=ret_per_full;
            impossible_pct_3yr_miroc6=impossible_pct_3yr;ret_per_full_3yr_miroc6=ret_per_full_3yr;
            
            np.savetxt(maindatadir+"impossiblepct_miroc6.csv",np.array(impossible_pct_miroc6),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_miroc6.csv",np.array(ret_per_full_miroc6),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"impossiblepct_3yr_miroc6.csv",np.array(impossible_pct_3yr_miroc6),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_3yr_miroc6.csv",np.array(ret_per_full_3yr_miroc6),delimiter=",",fmt='%10.2f')
        elif model==2:
            impossible_pct_mpige=impossible_pct;ret_per_full_mpige=ret_per_full;
            impossible_pct_3yr_mpige=impossible_pct_3yr;ret_per_full_3yr_mpige=ret_per_full_3yr;
            
            np.savetxt(maindatadir+"impossiblepct_mpige.csv",np.array(impossible_pct_mpige),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_mpige.csv",np.array(ret_per_full_mpige),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"impossiblepct_3yr_mpige.csv",np.array(impossible_pct_3yr_mpige),delimiter=",",fmt='%10.2f')
            np.savetxt(maindatadir+"retperfull_3yr_mpige.csv",np.array(ret_per_full_3yr_mpige),delimiter=",",fmt='%10.2f')

In [ ]:
#Convert vals to regional data for mapping (2 min)
#THIS AND THE CELL BELOW ARE NO LONGER NEEDED BECAUSE THE FIGURE IS CREATED IN MATLAB INSTEAD
if convertvalsformapping==1:
    impossible_pct_canesm5=np.loadtxt(maindatadir+"impossiblepct_canesm5.csv");
    impossible_pct_miroc6=np.loadtxt(maindatadir+"impossiblepct_miroc6.csv");
    impossible_pct_mpige=np.loadtxt(maindatadir+"impossiblepct_mpige.csv");
    ret_per_full_canesm5=np.loadtxt(maindatadir+"retperfull_canesm5.csv");
    ret_per_full_miroc6=np.loadtxt(maindatadir+"retperfull_miroc6.csv");
    ret_per_full_mpige=np.loadtxt(maindatadir+"retperfull_mpige.csv");

    impossiblearr_canesm5=region_ds.region;impossiblearr_miroc6=region_ds.region;
    impossiblearr_mpige=region_ds.region;
    retperarr_canesm5=region_ds.region;retperarr_miroc6=region_ds.region;
    retperarr_mpige=region_ds.region;
    for r in range(numregs):
        impossiblearr_canesm5=impossiblearr_canesm5.where(region_ds.region.values!=r,impossible_pct_canesm5[r])
        impossiblearr_miroc6=impossiblearr_miroc6.where(region_ds.region.values!=r,impossible_pct_miroc6[r])
        impossiblearr_mpige=impossiblearr_mpige.where(region_ds.region.values!=r,impossible_pct_mpige[r])
        retperarr_canesm5=retperarr_canesm5.where(region_ds.region.values!=r,ret_per_full_canesm5[r])
        retperarr_miroc6=retperarr_miroc6.where(region_ds.region.values!=r,ret_per_full_miroc6[r])
        retperarr_mpige=retperarr_mpige.where(region_ds.region.values!=r,ret_per_full_mpige[r])

In [ ]:
#Create map paralleling Thompson et al. 2023 Fig 4
#Runtime: 3 min
if maket23fig4==1:
    fig, axs = plt.subplots(3, 2, figsize=(20., 12.), dpi=80, num=None, subplot_kw={'projection': ccrs.PlateCarree()})

    for model in range(0,3):
        if model==0:
            impossiblearr=impossiblearr_canesm5;retperarr=retperarr_canesm5;
            ret_per_full=ret_per_full_canesm5;impossible_pct=impossible_pct_canesm5;
            mnamecap='CanESM5';leftlabel='a';rightlabel='b';
        elif model==1:
            impossiblearr=impossiblearr_miroc6;retperarr=retperarr_miroc6;
            ret_per_full=ret_per_full_miroc6;impossible_pct=impossible_pct_miroc6;
            mnamecap='MIROC6';leftlabel='c';rightlabel='d';
        elif model==2:
            impossiblearr=impossiblearr_mpige;retperarr=retperarr_mpige;
            ret_per_full=ret_per_full_mpige;impossible_pct=impossible_pct_mpige;
            mnamecap='MPI-ESM';leftlabel='e';rightlabel='f';

        row=model;

        axis_font = {'fontname':'Arial', 'size':'14'};

        # left subplot: impossible count for this model
        levels=[0, 10, 20, 30, 40, 50, 60]
        c = axs[row,0].contourf(lons,lats,impossiblearr,9,transform=ccrs.PlateCarree(),
                            cmap=mpl_cm.get_cmap('brewer_PuRd_09'), linestyle='solid',
                            levels=levels)
        cbar = plt.colorbar(c, shrink=0.7, ax=axs[row,0])
        cbar.set_label('% of ensemble members')
        cbar.outline.set_linewidth(0.5)
        axs[row,0].set_title(mnamecap+': Exceptional ensembles')
        axs[row,0].text(-190, 80, leftlabel,**axis_font);axs[row,0].coastlines();axs[row,0].set_ylim([-60, 90])

        # right subplot: ensemble return periods for this model
        from matplotlib import ticker, cm
        e = axs[row,1].contourf(lons,lats,impossiblearr,11,transform=ccrs.PlateCarree(),
                            colors='r',vmin=-10, vmax=1000) # might need adjusting
        levels=np.logspace(np.log10(10),np.log10(1000000000),5)
        d = axs[row,1].contourf(lons,lats,retperarr,6,transform=ccrs.PlateCarree(),
                            levels=levels, locator=ticker.LogLocator(base=10),
                            cmap=mpl_cm.get_cmap('Blues'), linestyle='solid',extend='max')
        cbar = plt.colorbar(d, shrink=0.7, ax = axs[row,1], ticks=levels)
        cbar.set_label('Years')
        cbar.outline.set_linewidth(0.5)
        axs[row,1].set_title(mnamecap+': Return periods')
        axs[row,1].text(-190, 80, rightlabel,**axis_font);axs[row,1].coastlines();axs[row,1].set_ylim([-60, 90])
        axs[row,1].text(-170, -50, 'Exceptional', fontsize = 10, 
                 bbox = dict(facecolor = 'red', alpha = 0.5))

        # Delete the Antarctic regions
        regs_in_ant = np.arange(170, 191)
        for each in regs_in_ant:
            ret_per_full.tolist().pop(each)
            impossible_pct.tolist().pop(each)

        # Percentages for this model
        imp = np.round(np.mean(np.asarray(impossible_pct)),2);
        full=np.round(sum(np.isnan(x) for x in ret_per_full)/numregs_noant*100,2);

        if model==0:
            imp_canesm5=imp;full_canesm5=full;
        elif model==1:
            imp_miroc6=imp;full_miroc6=full;
        elif model==2:
            imp_mpige=imp;full_mpige=full;


    plt.tight_layout()
    plt.savefig('modelsummary_t23f4.png', dpi=300)

In [ ]:
#Percent of ensemble members showing record events outside the GEV fit
#UNFINISHED -- USE CODE JUST ABOVE
#Runtime 5 sec
dothis=0;
if dothis==1:
    shapearr=np.zeros((numregs,));locarr=np.zeros((numregs,));scalearr=np.zeros((numregs,));
    thispdf=np.zeros((numregs,10000));

    era5data=np.loadtxt(maindatadir+'tw_regannmax_era5_adj.txt',delimiter=',').reshape((numregs,numyrs));
    canesm5data=np.loadtxt(maindatadir+'tw_regoverallmax_canesm5_adj.txt',delimiter=',').reshape((canesm5_enssz,numregs)); 
    miroc6data=np.loadtxt(maindatadir+'tw_regoverallmax_miroc6_adj.txt',delimiter=',').reshape((miroc6_enssz,numregs)); 
    mpigedata=np.loadtxt(maindatadir+'tw_regoverallmax_mpige_adj.txt',delimiter=',').reshape((mpige_enssz,numregs));

    pctexceeding_canesm5=np.zeros((numregs,))
    pctexceeding_miroc6=np.zeros((numregs,))
    pctexceeding_mpige=np.zeros((numregs,))

    for reg in range(numregs):
        ann_max_era5 = era5data[reg,:];
        res = gev.fit(ann_max_era5); # EVT distribution from obs
        shapearr[reg]=res[0];
        locarr[reg]=res[1];
        scalearr[reg]=res[2];

        x_val = np.linspace(np.min(ann_max_era5)-3, np.max(ann_max_era5)+3, 10000)
        tmp=gev.pdf(x_val, shapearr[reg], locarr[reg], scalearr[reg]);

        #Max non-zero GEV value (Celsius) -- reminder, this is still based on ERA5 adjusted to a GMST of 1C
        zeros=tmp[np.nonzero(tmp)]
        maxnonzerogevval=x_val[np.shape(zeros)[0]-1];

        #Percent of ensemble members exceeding the obs GEV fit
        pctexceeding_canesm5[reg]=100*np.sum(canesm5data[:,reg]>maxnonzerogevval)/canesm5_enssz;
        pctexceeding_miroc6[reg]=100*np.sum(miroc6data[:,reg]>maxnonzerogevval)/miroc6_enssz;
        pctexceeding_mpige[reg]=100*np.sum(mpigedata[:,reg]>maxnonzerogevval)/mpige_enssz;

In [ ]:
dosometroubleshooting=0;
if dosometroubleshooting==1:
    reg=233;var='tw';
    ann_max = load_annmax(reg,var)
    ann_max_new, GMST_new = remove_max(ann_max, GMST)
    offset = []
    a, b = np.polyfit(GMST_new, ann_max_new, 1)
    for i, each in enumerate(ann_max_new):
        actual = each
        predicted = GMST_new[i]*a +b
        offset.append(actual-predicted+(1*a+b))
    #max_val = np.max(offset) #the maximum observed
    shape, loc, scale = gev.fit(offset) # EVT distribution
    x_val = np.linspace(np.min(offset)-.5, np.max(offset)+2, 1000)
        #so e.g. for reg 51, which was most thoroughly troubleshooted, this pdf is defined from about 24.0 to 27.4 C
    dist_pdf = gev.pdf(x_val, shape, loc, scale)
    offset_full = []
    for i, each in enumerate(ann_max):
        actual = each
        predicted = GMST[i]*a +b
        offset_full.append(actual-predicted+(1*a+b))
    max_val = np.max(offset_full)
    chance = []
    for i, _ in enumerate(dist_pdf):
        P = []
        for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
            P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
        chance.append(sum(P)*100)
    x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
    if x == 0:
        result = 99999
    else:
        result = 100.*1/x

In [ ]:
#More troubleshooting follows

In [ ]:
#one_in_hundred(myarr):
shape, loc, scale = gev.fit(myarr)
x_val = np.linspace(np.min(myarr)-.5, np.max((myarr))+2, 1000)
#dist_pdf = gev.pdf(x_val, shape, loc, scale)
#ret_lev, chance = return_levels_plot(dist_pdf, x_val)
#return ret_lev[np.abs(np.asarray(chance)-1).argmin()]

In [ ]:
#obs_max_ret_per_removeevent(reg,var):
reg=68;
tmp_era5=np.loadtxt(maindatadir+'era5output_'+var+'_regions.txt',delimiter=',')[reg,:];
tmp_jra55=np.loadtxt(maindatadir+'jra55output_'+var+'_regions.txt',delimiter=',')[reg,:];
ann_max=(tmp_era5+tmp_jra55)/2;
#ann_max = load_annmax(reg,var)
ann_max_new, GMST_new = remove_max(ann_max, GMST)
offset = []
a, b = np.polyfit(GMST_new, ann_max_new, 1)
for i, each in enumerate(ann_max_new):
    actual = each
    predicted = GMST_new[i]*a +b
    offset.append(actual-predicted+(1*a+b)) #adjust for world with 1 deg of warming
#max_val = np.max(offset) # Maximum observed
shape, loc, scale = gev.fit(offset) # EVT distribution
x_val = np.linspace(np.min(offset)-.5, np.max(offset)+2, 1000)
dist_pdf = gev.pdf(x_val, shape, loc, scale)
offset_full = []
for i, each in enumerate(ann_max):
    actual = each
    predicted = GMST[i]*a +b
    offset_full.append(actual-predicted+(1*a+b))
max_val = np.max(offset_full)
chance = []
for i, _ in enumerate(dist_pdf):
    P = []
    for a, b in zip(dist_pdf[i:-1], dist_pdf[i+1:]):
        P.append(((a+b) / 2) * (x_val[1] - x_val[0])) 
    chance.append(sum(P)*100)
x = chance[np.abs(x_val - max_val).argmin()] # chance of observed max
if x == 0:
    result = 99999
else:
    result = 100.*1/x
return result

In [ ]:
era5regannmax_adj_tw=np.loadtxt(maindatadir+'era5output_tw_adj_regions.txt',delimiter=',');
jra55regannmax_adj_tw=np.loadtxt(maindatadir+'jra55output_tw_adj_regions.txt',delimiter=',');
oneinhundredval=np.zeros((numregs,));
for reg in range(numregs):
    era5arr=era5regannmax_adj_tw[reg,:];
    jra55arr=jra55regannmax_adj_tw[reg,:];
    myarr=(era5arr+jra55arr)/2;
    myrec=np.max(myarr);  
    oneinhundredval[reg]=one_in_hundred(myarr);

In [ ]:
regannmax=(era5regannmax_tw+jra55regannmax_tw)/2;
vals_retper_noant = [] #return periods
for each in np.arange(numregs):
    vals_retper_noant.append(obs_max_ret_per_removeevent(each,var)) #Return periods

In [36]:
whos

Variable                             Type              Data/Info
----------------------------------------------------------------
GMST                                 list              n=63
GMST_new                             ndarray           62: 62 elems, type `float64`, 496 bytes
GMST_since1981                       list              n=43
GMST_yr                              ndarray           63: 63 elems, type `int64`, 504 bytes
LogNorm                              type              <class 'matplotlib.colors.LogNorm'>
RegularGridInterpolator              type              <class 'scipy.interpolate<...>RegularGridInterpolator'>
a                                    float64           0.7623001584732692
abs_max                              function          <function abs_max at 0x3016c28e0>
actual                               float64           19.024
add_season_membership                function          <function add_season_membership at 0x132760180>
adjust_obs                      